### Stock predictions using different interpolation methods: 

Website used to extract data: 

https://www.wsj.com/market-data/quotes/index/DJIA/historical-prices

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve

%matplotlib inline

In [2]:
#Cubic Spline method
def CubicSpline(xvals,fvals,qvals):
    n = fvals.size - 1
    df = fvals[1:]-fvals[0:n]
    dx = xvals[1:]-xvals[0:n]
    dfdx = df/dx
    svals = np.zeros(qvals.size)
    
    rhs = dfdx[1:] - dfdx[0:n-1]
    diag = 2./3.*(dx[1:] + dx[0:n-1])
    
    data = np.array([diag,dx[1:]/3.,dx[0:n-1]/3.])
    dvals = np.array([0,-1,1])
    Bmat = spdiags(data, dvals, n-1, n-1)
    bvec = spsolve(Bmat,rhs)
    bvec = np.append(0.,bvec)
    
    cvec = dfdx - 2./3.*dx*bvec - dx/3.*np.append(bvec[1:],0.)
    avec = (dfdx - dx*bvec - cvec)/(dx**2.)
    
    for jj in range(1,n+1):
        
        indsr = qvals < xvals[jj] 
        indsl = qvals >= xvals[jj-1]
        inds = indsl*indsr
        
        dxloc = qvals[inds] - xvals[jj-1]
        svals[inds] = avec[jj-1]*dxloc**3. + bvec[jj-1]*dxloc**2. + cvec[jj-1]*dxloc + fvals[jj-1]
        
    return svals

In [3]:
#Lagrange L function 
def lfun(xvals,jj,x):
    lval = np.ones(x.size)
    n = xvals.size
    for l in range(0,n):
        if l != jj:
            lval *= (x-xvals[l])/(xvals[jj]-xvals[l])  
    return lval

#Lagrange Interperator 
def lagran_interp(xvals,fvals,x): #Lagrange
    n = fvals.size
    ipoly = np.zeros(x.size)
    for jj in range(0,n):
        ipoly += fvals[jj]*lfun(xvals,jj,x)
    return ipoly


#Newton's divided difference
def b1pl(xval, fval): #b1
    return (fval[1]-fval[0])/(xval[1]-xval[0])


def b2pl(xval, fval): #b2
    return (((fval[2]-fval[1])/(xval[2]-xval[1]))-b1pl(xval, fval))/(xval[2]-xval[0])

def b3pl(xval, fval): #b3
    f32 = (fval[3]-fval[2])/(xval[3]-xval[2])
    f21 = (fval[2]-fval[1])/(xval[2]-xval[1])
    f321 = (f32-f21)/(xval[3]-xval[1])
    return (f321-b2pl(xval, fval))/(xval[3]-xval[0])

def b4pl(xval, fval): #b4
    f43 = (fval[4]-fval[3])/(xval[4]-xval[3])
    f32 = (fval[3]-fval[2])/(xval[3]-xval[2])
    f432 = (f43-f32)/(xval[4]-xval[2])
    f21 = (fval[2]-fval[1])/(xval[2]-xval[1])
    f321 = (f32-f21)/(xval[3]-xval[1])
    f4321 = (f432-f321)/(xval[4]-xval[1])
    return (f4321-b3pl(xval, fval))/(xval[4]-xval[0])

def FourthOrder(xval, fval, x):
    m = 0
    b0 = fval[m]
    b1 = b1pl(xval, fval)*(x-xval[m])
    b2 = b2pl(xval, fval)*(x-xval[m])*(x-xval[m+1])
    b3 = b3pl(xval, fval)*(x-xval[m])*(x-xval[m+1])*(x-xval[m+2])
    b4 = b4pl(xval, fval)*(x-xval[m])*(x-xval[m+1])*(x-xval[m+2])*(x-xval[m+3])
    f6x = b0 + b1 + b2 + b3 + b4
    return f6x 

#data recived as file (download data from website above or use "wget") 
cvals = np.array([26925.05, 27480.03, 27847.66, 28390.18, 28323.40])
xval = np.linspace(0, 96, 5)
gval = np.array([65.5])


spline = CubicSpline(xval, cvals, gval) #spline
lagrange = lagran_interp(xval, cvals, gval) #lagrange
newton = FourthOrder(xval, cvals, gval) #Newton
print('Prediction using Spline is: %f' % np.max(spline))
print('Prediction using Lagrange is: %f' % np.max(lagrange))
print('Prediction using Newton is: %f' % np.max(newton))

Prediction using Spline is: 28277.565750
Prediction using Lagrange is: 28249.879961
Prediction using Newton is: 28249.879961


C:\Users\Daman\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
